In [1]:
from gensim.models import Word2Vec
import sys
reload(sys) 
sys.setdefaultencoding('UTF8')
import numpy as np

In [2]:
%time model_news = Word2Vec.load_word2vec_format("/home/ruoxu/opt/word2vec/training_result/GoogleNews-vectors-negative300.bin",binary=True)

In [207]:
%time model_aerospace = Word2Vec.load_word2vec_format("/home/ruoxu/opt/word2vec/training_result/vectors_aerospace.bin",binary=True)

In [208]:
X = []
y = []
words = []
for w in model_aerospace.vocab:
    if w in model_news.vocab:
        words.append(w)
        X.append(model_aerospace[w])
        y.append(model_news[w])

In [209]:
X = np.array(X)
y = np.array(y)
X.shape,y.shape,len(words)

((6236, 100), (6236, 300), 6236)

### Direct mapping

In [210]:
from keras.models import Sequential
from keras.layers.core import Dense, AutoEncoder,Activation
from keras.optimizers import Adadelta
from keras.layers.normalization import BatchNormalization

In [211]:
model = Sequential()
model.add(Dense(100,100))
model.add(Activation('tanh'))
model.add(Dense(100,300))
#model.add(BatchNormalization((300,),mode=1))
%time model.compile(loss='squared_hinge', optimizer='sgd')

In [212]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)
print X_train.shape,y_train.shape,X_test.shape,y_test.shape

In [213]:
#%time model.fit(X_train, y_train, batch_size=50, nb_epoch=20, show_accuracy=True, verbose=2, validation_data=(X_test, y_test))
#score = model.evaluate(X_test, y_test, show_accuracy=True, verbose=0)
%time model.fit(X, y, batch_size=50, nb_epoch=100, show_accuracy=True, verbose=2)
score = model.evaluate(X, y, show_accuracy=True, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [218]:
a = model.predict(X_test[:100])[10] 
b = y_test[:100][10]
np.sqrt(((a-b)**2).sum())
#np.dot(a,b) / np.sqrt((a**2).sum()) / np.sqrt((b**2).sum())

1.5747594941867875

In [215]:
(a**2).sum()

1.9722897678188658

### Nearest Neighbours

In [219]:
dissimilar_words = []
for w in words:
    nn_news = [nn[0] for nn in model_news.most_similar(w)]
    nn_aerospace = [nn[0] for nn in model_aerospace.most_similar(w)]
    if np.intersect1d(nn_news,nn_aerospace).shape[0] < 1:
        dissimilar_words.append(w)

KeyboardInterrupt: 

In [194]:
w,model_news.most_similar(w)

(u'played',
 [(u'playing', 0.764114499092102),
  (u'play', 0.6961715221405029),
  (u'Played', 0.6741452217102051),
  (u'plays', 0.6135597229003906),
  (u'Playing', 0.6080101728439331),
  (u'coached', 0.5274507999420166),
  (u'played_sparingly', 0.5190470218658447),
  (u'reffed', 0.5153231620788574),
  (u'starred', 0.5137808322906494),
  (u'competed', 0.511443018913269)])

In [220]:
dissimilar_words

[u'parameterization',
 u'localized',
 u'looking',
 u'electricity',
 u'adaptable',
 u'shielding',
 u'regional',
 u'experimentally',
 u'fractal',
 u'differentiated',
 u'volute',
 u'solid',
 u'succession',
 u'reliable',
 u'specially',
 u'scraper',
 u'budget',
 u'feasibility',
 u'contributed',
 u'designing',
 u'resilient',
 u'pioneering',
 u'specialist',
 u'agreement',
 u'reliably',
 u'here',
 u'reported',
 u'k',
 u'reports',
 u'expander',
 u'military',
 u'appropriately',
 u'topography',
 u'projection',
 u'classification',
 u'explained',
 u'brought',
 u'unit',
 u'vol',
 u'therefore',
 u'strike',
 u'until',
 u'standardized',
 u'relay',
 u'brings',
 u'glass',
 u'reversible',
 u'circumstances',
 u'intake',
 u'locked',
 u'pursue',
 u'concepts',
 u'plunger',
 u'example',
 u'wand',
 u'organized',
 u'want',
 u'absolute',
 u'preferably',
 u'travel',
 u'how',
 u'hot',
 u'hop',
 u'significance',
 u'preferable',
 u'activation',
 u'modulation',
 u'minimized',
 u'modest',
 u'monolithic',
 u'minimizes',

In [221]:
model_aerospace.most_similar('roller'),model_news.most_similar('roller')

([(u'tappet', 0.9273446202278137),
  (u'rocker', 0.924004077911377),
  (u'incorporates', 0.9158904552459717),
  (u'cams', 0.9060735702514648),
  (u'connections', 0.9029542207717896),
  (u'dismantling', 0.8963834047317505),
  (u'housing', 0.8942867517471313),
  (u'housed', 0.8930448889732361),
  (u'pushrods', 0.8927363157272339),
  (u'replaceable', 0.8902331590652466)],
 [(u'rollers', 0.7313302755355835),
  (u'golf_squash_karate', 0.5284586548805237),
  (u'slitters', 0.5193086862564087),
  (u'conveyor', 0.5072215795516968),
  (u'Roscoe_Parrish_muffed', 0.5034564733505249),
  (u'infeed', 0.5021519064903259),
  (u'hydraulically_driven', 0.49902409315109253),
  (u'coiler', 0.49209827184677124),
  (u'ballscrew', 0.480645090341568),
  (u'mandrels', 0.4805167317390442)])

## Summary
### 1. Mapping using MLP: 
####  The mapping does not work as expected. The distance between similar words and dissimilar words are almost the same
### 2. Intersection of nearest neighbours
#### The word density of small corpus and large corpus are different. Some words are similar in large corpus, but are not retrieved as NN, as there are words much more similar